# String encoding

### How it works?

* We replace strings by shorter code-words.
* Strings are searched in a dictionary, and the sequence of positions of the strings in the dictionary form the code-stream.

# 1. LZ77 [[Ziv and Lempel, 1977]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Ziv+Lempel+universal+sequential+data+compression+1977&btnG=)

* Jacov Ziv and Abraham Lempel proposed the LZ77 algorithm in 1977. 
* In the eighties, [LZSS](https://en.wikipedia.org/wiki/Lempel%E2%80%93Ziv%E2%80%93Storer%E2%80%93Szymanski) (a branch of LZ77) was
  implemented by Haruyasu Yoshizaki (in the [LHA compressor](https://en.wikipedia.org/wiki/LHA_(file_format)), discovering
  the possibilities of the LZ77 encoder.
* After that, a large number of text compressors have been based
  on LZ77 (or a variation of it). Some of the most famous
  are: [ARJ](https://en.wikipedia.org/wiki/ARJ), [RAR](https://en.wikipedia.org/wiki/RAR_(file_format), [gzip](https://en.wikipedia.org/wiki/Gzip) and [7z](https://en.wikipedia.org/wiki/7z).
* LZ77 processes a sequence of symbols using the structure:

<img src="data/LZ77.png" width="600">

* The dictionary and the look-ahead buffer have a fixed size and
  can be considered as a sliding window moving over the symbols while they are coded.
  In each iteration, the input of new
  symbols to the buffer generates the output of the oldest ones, which become the
  newest symbols of the dictionary.

### Encoder

1. Let $I$ the length of the dictionary and $J$ the length of the
   buffer.
2. Input the first $J$ symbols in the buffer.
3. While the input is not exhausted:
    1. Let $i$ the position of the first $j$
    symbols of the buffer in the dictionary, and $k$ the symbol that makes that $j$ can
    not be larger.
    2. Output $ijk$.
    3. Input the next $j+1$ symbols in the buffer.

### Example
<img src="data/LZ77_encoding_example.png" width="800">

### Decoder

1. While code-words $ijk$ are input:
    1. Output the $j$ symbols extracted from the position $i$ in the
    dictionary.
    2. Output $k$.
    3. Put all the decoded symbols in the beginnig of the buffer.

### Example
<img src="data/LZ77_decoding_example.png" width="500">

* Parameters $I$ and $J$ control the performance
  of the algorithm. They should be large enough to guarantee the
  matching of long strings, but should be small in order to reduce
  the number of bits of the code-words $ijk$. Typical sizes are:
  $\log_2(I)=12.0$ and $\log_2(J)=4.0$.

### Lab

In [1]:
import math
from bitarray import bitarray


class LZ77Compressor:
    """
    A simplified implementation of the LZ77 Compression Algorithm
    """
    MAX_WINDOW_SIZE = 400

    def __init__(self, window_size=20):
        self.window_size = min(window_size, self.MAX_WINDOW_SIZE)  
        self.lookahead_buffer_size = 15 # length of match is at most 4 bits

    def compress(self, input_file_path, output_file_path, verbose=True):
        data = None
        #data = 'mahi magi magi mahi mahi hello mahi madi mahi mahi mahi facebook hello world mahi magi magi mahi mahi hello mahi madi mahi mahi mahi facebook hello world mahi magi magi mahi mahi hello mahi madi mahi mahi mahi facebook hello world '
        #data = input_data
        i = 0
        output_buffer = bitarray(endian='big')

        # read the input file 
        try:
            with open(input_file_path, 'r') as input_file:
                data = input_file.read()
        except IOError:
            print ('Could not open input file ...')
            raise

        while i < (len(data)-1):
            #print i

            match = self.findLongestMatch(data, i)

            if match: 
                # Add 1 bit flag, followed by 12 bit for distance, and 4 bit for the length
                # of the match 
                (bestMatchDistance, bestMatchLength) = match

                output_buffer.append(True)
                output_buffer.frombytes(bytes(chr(bestMatchDistance >> 4),"latin-1"))
                output_buffer.frombytes(bytes(chr(((bestMatchDistance & 0xf) << 4) | bestMatchLength),"latin-1"))

                if verbose:
                    print(("<1, %i, %i>") % (bestMatchDistance, bestMatchLength), end=' ')

                i += bestMatchLength

            else:
                # No useful match was found. Add 0 bit flag, followed by 8 bit for the character
                output_buffer.append(False)
                output_buffer.frombytes(bytes(data[i],"latin-1"))

                if verbose:
                    print(("<0, %s>") % data[i], end=' ')

                i += 1

        # fill the buffer with zeros if the number of bits is not a multiple of 8		
        output_buffer.fill()

        # write the compressed data into a binary file if a path is provided
        if output_file_path:
            try:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(output_buffer.tobytes())
                    print ("\n" + "File was compressed successfully and saved to output path ...")
                    return None
            except IOError:
                print ('Could not write to output file path. Please check if the path is correct ...')
                raise

        # an output file path was not provided, return the compressed data
        return output_buffer


    def decompress(self, input_file_path, output_file_path):
        data = bitarray(endian='big')
        output_buffer = []

        # read the input file
        try:
            with open(input_file_path, 'rb') as input_file:
                data.fromfile(input_file)
        except IOError:
            print ('Could not open input file ...')
            raise

        while (len(data)-1) >= 9:

            flag = data.pop(0)

            if not flag:
                byte = data[0:8].tobytes()

                output_buffer.append(byte)
                del data[0:8]
            else:
                byte1 = ord(data[0:8].tobytes())
                byte2 = ord(data[8:16].tobytes())

                del data[0:16]
                distance = (byte1 << 4) | (byte2 >> 4)
                length = (byte2 & 0xf)

                for i in range(length):
                    output_buffer.append(output_buffer[-distance])
        out_data =  b''.join(output_buffer)

        if output_file_path:
            try:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(out_data)
                    print ('File was decompressed successfully and saved to output path ...')
                    return None 
            except IOError:
                print ('Could not write to output file path. Please check if the path is correct ...')
                raise 
        return out_data

    def findLongestMatch(self, data, current_position):
        end_of_buffer = min(current_position + self.lookahead_buffer_size, len(data) + 1)

        best_match_distance = -1
        best_match_length = -1

        # Optimization: Only consider substrings of length 2 and greater, and just 
        # output any substring of length 1 (8 bits uncompressed is better than 13 bits
        # for the flag, distance, and length)
        for j in range(current_position + 2, end_of_buffer):

            start_index = max(0, current_position - self.window_size)
            substring = data[current_position:j]

            for i in range(start_index, current_position):

                repetitions = int(len(substring) / (current_position - i))

                last = int(len(substring) % (current_position - i))

                matched_string = data[i:current_position] * repetitions + data[i:i+last]

                if matched_string == substring and len(substring) > best_match_length:
                    best_match_distance = current_position - i 
                    best_match_length = len(substring)

        if best_match_distance > 0 and best_match_length > 0:
            return (best_match_distance, best_match_length)
        
        return None
    
if __name__ == "__main__":  
    compressor = LZ77Compressor(window_size=20) # window_size is optional

    #Read from a file and Write to a file
    input_file_path = 'input.txt'
    output_file_path = 'output.txt'
    result_file_path = 'result.txt'
    
    compressed_data = compressor.compress(input_file_path,output_file_path) #Compress the "input_file"
    decompressed_data = str(compressor.decompress(output_file_path,result_file_path))#Decompress the "output_file"

    print ("\n")
    with open(input_file_path, 'r') as input_file: #Read a file and save the result in the variable "original_data"
                original_data = input_file.read()
                original_data = original_data[:-1] #Delete the last character, \n

    with open(result_file_path, 'r') as input_file: #Read a file and save the result in the variable "result_data"
                result_data = input_file.read()
    
    print("The content of the first file is:" , original_data)
    print("The content of the decompress file is:" , result_data)

    if original_data == result_data:
        result = True
    else:
        result = False

    print("Are both files similar?", result)

ModuleNotFoundError: No module named 'bitarray'

## 2. LZ78 [[Ziv and Lempel, 1978]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Ziv+Lempel+1978&btnG=)

* In 1978, Ziv and Lempel published the [LZ78 algorithm](https://en.wikipedia.org/wiki/LZ77_and_LZ78).

* LZ78 represents the dictionary in a recursive way with the idea
  of reducing the memory used for representing the strings in the dictionary. Now, each
  entry in the dictionary is a pair $wk$, where $w$ is an index to
  an entry of the dictionary and $k$ is a symbol. In fact, each pair $wk$
  represents the string that results from the concatenation of the string
  $w$ and the symbol $k$, where $w$ can be recursively computed in the same way
  we have found $wk$.
  
* We will denote the string that $w$ represents by *string*$(w)$.
  
* The empty string is obtained by *string*$(0)$.

### Encoder

1. $w\leftarrow 0$.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $wk$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow 0$.

### Example
<img src="data/LZ78_encoding_example.png" width="700">

### Decoder

1. While the input is not exhausted:
    1. Input $wk$.
    2. Output $\text{string}(w)$.
    3. Output $k$.
    4. Insert $wk$ in the dictionary.

### Example
<img src="data/LZ78_decoding_example.png" width="600">

### Lab

In [2]:
import io

def encoder(uncompressed):
    """Compress a string to a list of output symbols."""

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}

    w = ""
    result = []
    for k in uncompressed:
        wk = w + k
        if wk in dictionary:
            w = wk
        else:
            result.append(dictionary[w])
            # Add wk to the dictionary.
            dictionary[wk] = dict_size
            dict_size += 1
            w = k

    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result


def decoder(compressed):
    """Decompress a list of output ks to a string."""

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}

    # use io.StringIO(), otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = io.StringIO()
    w = compressed.pop(0)
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)

        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry
    return result.getvalue()


# Testing the algorithm
cadena = "Hola, soy Claudio y este algoritmo es correcto..."
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Chains in which we can see this algorithm compresses much more
cadena = "AABBAAABBBAAAABBBBAAAAABBBBBABABA"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Another example
cadena = "TOBEORNOTTOBEORTOBEORNOT"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

['H', 'o', 'l', 'a', ',', ' ', 's', 'o', 'y', ' ', 'C', 258, 'u', 'd', 'i', 'o', ' ', 264, 'e', 's', 't', 'e', ' ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'm', 271, 274, ' ', 'c', 282, 'r', 'e', 'c', 't', 'o', '.', 297]
Hola, soy Claudio y este algoritmo es correcto...

['A', 'A', 'B', 'B', 256, 257, 258, 260, 261, 262, 263, 258, 267, 257, 269]
AABBAAABBBAAAABBBBAAAAABBBBBABABA

['T', 'O', 'B', 'E', 'O', 'R', 'N', 'O', 'T', 256, 258, 260, 265, 259, 261, 263]
TOBEORNOTTOBEORTOBEORNOT


## 3. LZW [[Welch, 1984]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Terry+Welch+1984&btnG=)

* In 1984, Terry A. Welch proposed the [LZW algorithm](https://en.wikipedia.org/wiki/Lempel%E2%80%93Ziv%E2%80%93Welch),
  an improved version of the LZ78 algorithm that does not
  writes raw symbols ($k$ fields) to the code-stream.

* LZW was selected as the encoding engine for the [GIF (Graphics
  Interchange Format)](https://en.wikipedia.org/wiki/GIF), and for the compressor [`compress`](https://en.wikipedia.org/wiki/Compress).
  
* Initially, the dictionary is filled with the $2^k$ possible
  symbols (*roots*), that are stored in the first entries (for 1-byte symbols: $0\cdots255$).

### Encoder

1. $w\leftarrow$ first input symbol.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $\leftarrow w$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow k$.

### Example
<img src="data/LZW_encoding_example.png" width="800">

### Decoder

1. $c\leftarrow$ first input code-word.
2. Output $c$.
3. $c'\leftarrow c$.
4. While the input is not exhausted:
    1. $c\leftarrow$ next input code-word.
    2. $w\leftarrow c'$.
    3. If $c$ is found in the dictionary, then:
        1. Output string$(c)$.
    4. Else:
        1. Output string$(w)$.
        2. Output $k$.
    5. $k\leftarrow$ first symbol of the last output.
    6. Insert $wk$ in the dictionary.
    7. $c'\leftarrow c$.

### Example
<img src="data/LZW_decoding_example.png" width="600">

### Lab

In [4]:
# https://rosettacode.org/wiki/LZW_compression#Python

# TO-DO
import io

def encoder(uncompressed):
    """Compress a string to a list of output symbols."""
 
    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
 
    w = ""
    result = []
    for k in uncompressed:
        wk = w + k
        if wk in dictionary:
            w = wk
        else:
            result.append(dictionary[w])
            # Add wk to the dictionary.
            dictionary[wk] = dict_size
            dict_size += 1
            w = k
 
    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result
 
 
   
def decoder(compressed):
    """Decompress a list of output ks to a string."""
     
    # Build the dictionary.
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}
 
    # use io.StringIO(), otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = io.StringIO()
    w = chr(compressed.pop(0))
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)
 
        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
 
        w = entry
    return result.getvalue()
 

# Testing the algorithm
cadena = "Hola, soy Claudio y este algoritmo es correcto..."
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Other examples
cadena = "AABBAAABBBAAAABBBBAAAAABBBBBABABA"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

cadena = "TOBEORNOTTOBEORTOBEORNOT"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

[72, 111, 108, 97, 44, 32, 115, 111, 121, 32, 67, 258, 117, 100, 105, 111, 32, 264, 101, 115, 116, 101, 32, 97, 108, 103, 111, 114, 105, 116, 109, 271, 274, 32, 99, 282, 114, 101, 99, 116, 111, 46, 297]
Hola, soy Claudio y este algoritmo es correcto...

[65, 65, 66, 66, 256, 257, 258, 260, 261, 262, 263, 258, 267, 257, 269]
AABBAAABBBAAAABBBBAAAAABBBBBABABA

[84, 79, 66, 69, 79, 82, 78, 79, 84, 256, 258, 260, 265, 259, 261, 263]
TOBEORNOTTOBEORTOBEORNOT
